# Data pipeline for uniform FEEDBACK - Herschel - Spitzer data set
- Works with data from Nextcloud repository
- Objective: get all Herschel and Spitzer data with a uniform resolution, grid and file nomenclature -set by the FEEDBACK [CII] data- for easy analysis

In [1]:
import numpy as np
import astropy.io.fits as pyfits
import astropy.wcs as wcs

from reproject import reproject_interp
from scipy.ndimage import gaussian_filter

import os
from os import listdir
from os.path import isfile, join
import time

In [2]:
#### User input ####
## region
region_name = "RCW79"
path_name = "../data_CIIcorr/data_FEEDBACK/original/{region}/".format(region = region_name) ## directory where to get the original data
path_intermediate = "../data_CIIcorr/data_FEEDBACK/intermediate/" ## directory where to store intermediate results of the processing
path_final = "../data_CIIcorr/data_FEEDBACK/{region}/".format(region = region_name) ## directory where to get the final data products will be stored

## Native resolution of Spitzer and Herschel data
#native_res_list = [10.7, 5.6, 17.6, 23.9, 35.2, 1.7, 1.77, 1.88, 1.98] ## first 5: Herschel, last five: Spitzer
#name_info_list = ["Herschel_PACS_160",  ## first 5: Herschel, last five: Spitzer
#             "Herschel_PACS_70",
#             "Herschel_SPIRE_250",
#             "Herschel_SPIRE_350",
#             "Herschel_SPIRE_500",
#             "Spitzer_IRAC_3p6",
#             "Spitzer_IRAC_4p5",
#             "Spitzer_IRAC_5p8",
#             "Spitzer_IRAC_8",
#            ]
#do_corr = False

## for the Herschel 100 micron data
native_res_list = [7.0] 
name_info_list = ["Herschel_PACS_100"]
do_corr = True
corr_facts = [1.66e4]

## info for the [CII] data
info_CII = 'upGREAT_CII'

## initial resolution of the [CII] data
init_res_CII = 20.

## Target resolution of the final data
#target_res = 20. ## arcsec
target_res = 36. ## arcsec (the lowest Herschel resolution)
#smoothRes = 45. ## arcsec (preparation for GUSTO)

########################

In [3]:
## Collect the Herschel and Spitzer data
file_names_dir = [f for f in listdir(path_name) if isfile(join(path_name, f))]

## select files of interest and align with the data file
file_names = []
for name in name_info_list:
    res = name.split("_")
    for fname in file_names_dir:
        fres = fname.split("_")
        if(res[-1] == fres[3]):
            file_names.append(fname)
            
## Ensure that everything worked out well
assert len(file_names) == len(name_info_list)

## print names and resolution for verification
#file_names = sorted(file_names)
for i, name in enumerate(file_names):
    print("The assumed native resolution = {res}, for file {name} \n".format(res = native_res_list[i], name = name))

The assumed native resolution = 7.0, for file RCW79_Herschel_PACS_100_large_centered.fits 



### End of what should be verified when running the script

### Functions to be called in the script

In [4]:
def get_data_header(path, file_name):
    ## get the data and header
    hdu = pyfits.open(path + file_name)
    hdu.info()
    data, header = hdu[0].data, hdu[0].header
    
    ## handle RCW 79 file at 100 micron which is not uniform compared to the rest of the data
    name_res = file_name.split("_")
    if(name_res[0] == 'RCW79' and name_res[3] == '100'):
        data, header = hdu[1].data, hdu[1].header
    
    return data, header

In [5]:
def smooth_data(data, sigma_conv):
    V = data.copy()
    V[np.isnan(data)] = 0
    VV = gaussian_filter(V, sigma=sigma_conv)
    
    W = 0*data.copy() + 1
    W[np.isnan(data)] = 0
    WW = gaussian_filter(W,sigma=sigma_conv)

    return VV/WW

#### End of function definitions

### Smoothing the [CII] data

In [6]:
## open the FEEDBACK [CII] fits file
hdu = pyfits.open('../data_CIIcorr/data_FEEDBACK/original/CII_maps/{region}_CII_final_res{resCII}_grid8_0p5_clean_integrated.fits'.format(region = region_name,
                                                                                                                              resCII = int(init_res_CII + 0.5)
                                                                                                                                 ))
hdu.info()
dataCII = hdu[0].data
headerCII = hdu[0].header

Filename: ../data_CIIcorr/data_FEEDBACK/original/CII_maps/RCW79_CII_final_res20_grid8_0p5_clean_integrated.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      36   (188, 188)   float32   


In [7]:
## smooth the CII data ##

## the pixel size in arcseconds and the Gaussian convolution kernel
pix_size_CII = headerCII['CDELT2']*3600. ## 3600 to convert from degree units to arcsecond units
sigma_conv_CII = np.sqrt((target_res/2.35)**2 - (init_res_CII/2.35)**2) / pix_size_CII

## get the smoothed data
data_sm_CII = smooth_data(dataCII, sigma_conv_CII)

## update the CII header
headerCII['BMAJ'] = target_res / 3600.
headerCII['BMIN'] = target_res / 3600.

## store the smoothed [CII] 
if(os.path.exists(path_final) == False):
    os.mkdir(path_final)
    
new_hdu_CII = pyfits.PrimaryHDU(data_sm_CII, headerCII)
new_hdu_CII.writeto("{path}/{region}_{info}_{res}_{grid}_integrated.fits".format(path = path_final, 
                                                                      region = region_name, 
                                                                      info = info_CII, 
                                                                      res = int(target_res + 0.5), 
                                                                      grid = int(pix_size_CII + 0.5)
                                                                     ), 
                    overwrite = True)

/var/folders/pb/xs3f5mtd5bq7lmrql1dj8p040000gn/T/ipykernel_2090/638087385.py:10: RuntimeWarning: invalid value encountered in divide
  return VV/WW


### Smoothing and regridding the Herschel/Spitzer data

In [8]:
## open the FEEDBACK [CII] fits file
#hdu = pyfits.open('../data/data_FEEDBACK/original/CII_maps/{region}_CII_final_res36_grid8_0p5_clean_integrated.fits'.format(region = region_name))
hdu = pyfits.open("{path}/{region}_{info}_{res}_{grid}_integrated.fits".format(path = path_final, 
                                                                      region = region_name, 
                                                                      info = info_CII, 
                                                                      res = int(target_res + 0.5), 
                                                                      grid = int(pix_size_CII + 0.5)
                                                                     ))
hdu.info()
dataCII = hdu[0].data
headerCII = hdu[0].header
wCII = wcs.WCS(headerCII)

Filename: ../data_CIIcorr/data_FEEDBACK/RCW79//RCW79_upGREAT_CII_36_8_integrated.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      36   (188, 188)   float32   


In [9]:
## loop over all data files. First regrid and then smooth the data to the target resolution and grid
for i, file_name in enumerate(file_names):
    ## get the data file and header information
    data, header = get_data_header(path_name, file_name)
    
    if(do_corr):
        data = data*corr_facts[i]
        header['BUNIT'] = 'MJy/sr'
    
    ## get the pixel size in arcseconds
    pix_size = header["CDELT2"]*3600. ## 3600 to convert from degree units to arcsecond units
    
    #### first smooth the data ####
    if(native_res_list[i] < target_res):
        start_time = time.time()
        print("Smoothing the data of {file} with an assumed native resolution {res}".format(file = file_name, res = native_res_list[i]))
        
        ## calculate the needed Gaussian convolution kernel in pixel size values
        sigma_conv = np.sqrt((target_res/2.35)**2 - (native_res_list[i]/2.35)**2) / pix_size ## /2.35 FWHM -> sigma
        
        ## smooth the data
        data_smooth = smooth_data(data, sigma_conv)
        
        ## store the smoothed data in the intermediate directory
        new_hdu = pyfits.PrimaryHDU(data_smooth, header)
        new_hdu.writeto("{path}smoothed_{file}".format(path = path_intermediate, file = file_name), overwrite = True)
        
        end_time = time.time()
        print(" -> Done in a total run time of {time} seconds \n".format(time = end_time - start_time))
    else:
        print("No smoothing done \n")
        new_hdu = pyfits.PrimaryHDU(data, header)
        new_hdu.writeto("{path}smoothed_{file}".format(path = path_intermediate, file = file_name), overwrite = True)
        
    
    #### perform the regridding on the smoothed data ####
    start_time = time.time()
    print("Starting the regridding on {file}".format(file = file_name))
    
    ## open HDU to perform regridding on
    hdu = pyfits.open("{path}smoothed_{file}".format(path = path_intermediate, file = file_name))
    
    ## perform reprojection on the [CII] grid
    array, footprint = reproject_interp(hdu, headerCII)
    
    ## prepare the header to store the regridded map
    new_header = headerCII.copy()
    new_header['BUNIT'] = 'MJy/sr'
    new_header['LINE'] = name_info_list[i]
    
    ## save the regridded map
    new_hdu = pyfits.PrimaryHDU(array, new_header)
    if(os.path.exists(path_final) == False):
        os.mkdir(path_final)
    new_hdu.writeto("{path}/{region}_{info}_{res}_{grid}.fits".format(path = path_final, region = region_name, info = name_info_list[i], 
                                                                      res = int(target_res + 0.5), grid = int(headerCII["CDELT2"]*3600. + 0.5)
                                                                     ))
    
    end_time = time.time()
    print(" -> Done in a total run time of {time} seconds \n".format(time = end_time - start_time))
    print(" ")
        
    
    

Filename: ../data_CIIcorr/data_FEEDBACK/original/RCW79/RCW79_Herschel_PACS_100_large_centered.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     286   ()      
  1  image         1 ImageHDU        52   (1659, 1716)   float64   
  2  coverage      1 ImageHDU        55   (1659, 1716)   float64   
  3  stDev         1 ImageHDU        55   (1659, 1716)   float64   
  4  History       1 ImageHDU        23   ()      
  5  HistoryScript    1 BinTableHDU     39   81R x 1C   [321A]   
  6  HistoryTasks    1 BinTableHDU     46   55R x 4C   [1K, 38A, 1K, 9A]   
  7  HistoryParameters    1 BinTableHDU     74   507R x 10C   [1K, 21A, 7A, 31A, 1L, 1K, 1L, 58A, 11A, 35A]   
Smoothing the data of RCW79_Herschel_PACS_100_large_centered.fits with an assumed native resolution 7.0
 -> Done in a total run time of 0.23305797576904297 seconds 

Starting the regridding on RCW79_Herschel_PACS_100_large_centered.fits
 -> Done in a total run time of 0.029017210